In [1]:
import json
import pandas as pd
import gensim
import gensim.downloader as api
from sklearn.utils import shuffle
import zipfile
import sys
import requests, io
import re 
import random
import numpy as np
import matplotlib.pyplot as plt

import json
import pandas as pd
import pymorphy2
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import gensim.models as models
from gensim.models import Word2Vec
from gensim.models import FastText
from gensim.models import KeyedVectors
import gensim.downloader as api
import sentence_transformers
from sentence_transformers import SentenceTransformer
from sklearn.utils import shuffle
import zipfile
import sys
import requests, io
import re 
import random
import numpy as np
import matplotlib.pyplot as plt
from nltk.tokenize import sent_tokenize
# from google.colab import files

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Andrey\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Andrey\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
def get_states(sim, df, match_threshold):
    (TP, FP, FN, TN) = (0, 0, 0, 0)
    for i in range(len(sim)):
        if df['need_match'][i:i+1].item():
            if sim[i] >= match_threshold: 
                TP += 1
            else:
                FN += 1
        else:
            if sim[i] >= match_threshold: 
                FP += 1
            else:
                TN += 1
    
    return (TP, FP, FN, TN)
        

def calc_f1_score(sim, df, match_threshold):
    (TP, FP, FN, TN) = get_states(sim, df, match_threshold)
#     print(TP, FP, FN, TN)
    return round(float(2*TP / (2*TP + FP + FN)), 3)

import re


def sent_preprocess(text:str):
    preprocessed_text = sent_tokenize(text)
    for i in range(len(preprocessed_text)):
        res = re.sub(r'([^\w\s])|([0-9]+)', '', preprocessed_text[i])
        # res = re.sub(r'', '' ,  res)
        preprocessed_text[i] = (preprocessed_text[i], res)
    
    preprocessed_text = list(filter(lambda sentence: sentence[0] != '', preprocessed_text))
    
    return preprocessed_text

In [4]:
match_df_path = './data/match-1.json'
unmatch_df_path = './data/unmatch.json'

In [5]:
match_df = pd.read_json(match_df_path)
unmatch_df = pd.read_json(unmatch_df_path)

In [ ]:
match_df

In [2]:
model_ = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

In [7]:
sentences1 =sent_preprocess(match_df["text_rp"][0])

In [33]:
!pip install pymystem3

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com/


## Базовое разбиение текста на предложения

In [43]:
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

delete = "[\xa0|_|&]"

def sent_preprocess(text:str):
    preprocessed_text = sent_tokenize(text)

    for i in range(len(preprocessed_text)):
        res = preprocessed_text[i].translate(str.maketrans('', '', punctuation))
        res = re.sub(delete,"", res)
        preprocessed_text[i] = res
    
    preprocessed_text = list(filter(lambda sentence: len(sentence.split(' ')) >= 2, preprocessed_text))
    
    return preprocessed_text

In [44]:
sent_preprocess(match_df["text_rp"][0])

['Данный курс знакомит слушателей с основами управления беспилотными автомобилями',
 'Рассматриваются основные компоненты таких сиcтем модели и методы восприятия информации при помощи сенсоров или человекомашинного интерфейса методы определения собственного положения на известной и неизвестной карте методы планирования пути в том числе на графах и не на графах фреймворк подходящий для разработки собственных решений для беспилотных транспортных средств',
 'Данный курс знакомит с уровнями автономности беспилотных средств с основами машинного обучения в том числе при помощи нейросетей операционной системой Robot Operating System являющейся дефакто стандартным решением для многих роботов в том числе бепилотных автомобилей',
 'Курс позволяет освоить основные приёмы программирования в ROS алгоритмов локализации и построения карты SLAM алгоритмов машинного зрения ',
 'Сформировать у обучающихся знания умения и навыки обоснованного и результативного применения существующих методов моделей а та